In [1]:
import sys
sys.path.append("../")

from rdflib import ConjunctiveGraph, URIRef
import requests
from requests.exceptions import InvalidURL
import random
import json
from tqdm.notebook import tqdm
import pandas as pd
from metrics.WebResource import WebResource

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST driver version for 113.0.5672
[WDM] - Driver [/Users/gaignard-a/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache


In [12]:
def get_live_deploys_urls():
    results = []
    live_deploys_remote_file = "https://raw.githubusercontent.com/BioSchemas/bioschemas.github.io/master/_data/live_deployments.json"
    res = requests.get(live_deploys_remote_file)
    live_deploys = res.json()
    
    for r in live_deploys["resources"]:
        for p in r["profiles"]:
            if "exampleURL" in p.keys():
                results.append(p["exampleURL"])
            else : 
                print(f"no example URL for profile {p['profileName']} and url {r['url']}")
    print(f"found {len(set(results))} live deploys")
    return list(set(results))



def test_ld_bioschemas_annot():
        res = get_live_deploys_urls()
        errors = []
        results = []
        
        i = 0
        for r in tqdm(res):
        #for r in reversed(res):
            row = {}
            status_code = requests.head(r).status_code
            print(r)
            row['url'] = r
            print("Status code: " + str(status_code))
            row['status code'] = str(status_code)
            try :
                kg = WebResource(r).get_rdf()
                print("Triples: " + str(len(kg)))
                row['Retrieved triples'] = len(kg)
                if not len(kg) > 0:
                    print(f"# Error with {r}")
                    row['Error'] = True
                    errors.append(r)
            except UnicodeDecodeError as err:
                print(err)
            results.append(row)

        print(f"{len(res)} tested URLS")
        print(f"{len(errors)} failing URLS")
        print(errors)
        df = pd.read_json(json.dumps(results))
        return df
    
def retrieve_bioschemas(url):  
    #FC_get_md = "https://fair-checker.france-bioinformatique.fr/api/inspect/get_rdf_metadata"
    FC_get_md = "http://localhost:5000/api/inspect/get_rdf_metadata"
    kg = ConjunctiveGraph()
    res = requests.get(url=FC_get_md, params={"url": url})
    try:
        kg.parse(data=res.text, format="json-ld")
    except Exception as e:
            print(e)
    print(f"Loaded {len(kg)} RDF triples from {url}")
    return kg

def check_bioschemas(url):  
    #FC_bs_valid = "https://fair-checker.france-bioinformatique.fr/api/inspect/bioschemas_validation"
    #FC_bs_valid = "https://fair-checker.france-bioinformatique.fr/api/inspect/bioschemas_validation_by_conformsto"
    FC_bs_valid = "http://localhost:5000/api/inspect/bioschemas_validation_by_conformsto"
    res = requests.get(url=FC_bs_valid, params={"url": url})
    #print(res.text)
    #try:
    #    print(json.dumps(res.json(), indent=True))
    #except Exception as e:
    #    print(e)
    return res.json()

def has_conforms_to(kg):
    conformsTo = URIRef("http://purl.org/dc/terms/conformsTo")
    return (None, conformsTo, None) in kg

In [3]:
lds = get_live_deploys_urls()
#lds = random.sample(lds,20)
print(f"retrieved {len(lds)} live deploys")

found 136 live deploys
retrieved 136 live deploys


In [188]:
rows = []
i = 0
for ld in tqdm(lds):
    row = {"URL": ld}    
    try :
        status_code = requests.head(ld).status_code
        row["HTTP_status"] = status_code
        kg = retrieve_bioschemas(ld)
        #print(kg.serialize(format="ntriples"))
        row["nb_triples"] = len(kg)
        row["has_conforms_to"] = has_conforms_to(kg)
    except InvalidURL as e :
        print(f"Invalid url: {ld}")
    rows.append(row)
    
df1 = pd.DataFrame.from_records(rows)
df1.to_csv("bioschemas_harvesting.csv")
df1

  0%|          | 0/136 [00:00<?, ?it/s]

Loaded 14 RDF triples from https://bridgedb.github.io/data/gene_database/
Loaded 95 RDF triples from http://itsonedb.cloud.ba.infn.it/
Loaded 124 RDF triples from https://www.sib.swiss/training/course/20230426_DOCK
Loaded 0 RDF triples from https://proteinensemble.org/PED00014
Loaded 25 RDF triples from https://bioregistry.io/registry/hgnc
Loaded 373 RDF triples from https://humanmine.org/
Loaded 17 RDF triples from https://modelarchive.org/
Loaded 13 RDF triples from https://www.orpha.net/consor/cgi-bin/OC_Exp.php?Expert=141189&lng=en
Loaded 0 RDF triples from https://deb-central.org/
Loaded 34 RDF triples from https://bio.tools/blast
Loaded 10 RDF triples from https://scholia.toolforge.org/chemical/Q18216
Loaded 36 RDF triples from https://bmrb.io/data_library/summary/index.php?bmrbId=30309
Loaded 26 RDF triples from https://bioschemas.org/tutorials/what_why_bioschemas
Loaded 0 RDF triples from https://proteinensemble.org/
Loaded 28 RDF triples from http://edgar.biocomp.unibo.it/
Loa

Loaded 20 RDF triples from https://www.elixir-europe.org/events/biohackathon-2018-paris
Expecting value: line 1 column 1 (char 0)
Loaded 0 RDF triples from https://pippa.psb.ugent.be/pippa_experiments/consult_experiment_basic_info/55
Loaded 10 RDF triples from https://psnpbind.org/protein/1owh
Loaded 44 RDF triples from http://www.cathdb.info
Loaded 0 RDF triples from https://wikipathways.org/
Loaded 35 RDF triples from https://bioschemas.org/meetings/2022-01_SWAT4HCLS_leiden
Loaded 19 RDF triples from https://www.omicsdi.org/#/dataset/arrayexpress-repository/E-MTAB-6848
Loaded 0 RDF triples from https://www.gbif.org/species/5220113
Loaded 48 RDF triples from http://phenpath.biocomp.unibo.it/phenpath/
Loaded 0 RDF triples from http://159.149.160.88/pscan_chip_dev/
Loaded 27 RDF triples from https://identifiers.org/
Loaded 31 RDF triples from https://www.ensembl.org/
Loaded 49 RDF triples from https://zbmed-semtec.github.io/dome-galaxy-training/docs/DOME.html
Loaded 46 RDF triples from 

,URL,HTTP_status,nb_triples,has_conforms_to
0,https://bridgedb.github.io/data/gene_database/,301.0,14.0,True
1,http://itsonedb.cloud.ba.infn.it/,200.0,95.0,False
2,https://www.sib.swiss/training/course/20230426...,200.0,124.0,True
3,https://proteinensemble.org/PED00014,200.0,0.0,False
4,https://bioregistry.io/registry/hgnc,200.0,25.0,True
...,...,...,...,...
131,https://www.guidetopharmacology.org/,200.0,44.0,False
132,https://string-db.org/,200.0,182.0,True
133,https://ega-archive.org/datasets/EGAD00000000001,200.0,25.0,False
134,https://tess.elixir-europe.org/events/1st-inte...,200.0,35.0,True


In [13]:
rows = []
#print(lds)
for ld in tqdm(lds):
    try: 
        bs_valid = check_bioschemas(ld)
        for entity in bs_valid.keys():
            row = {"Bioschemas Live Deploy URL": ld}  
            row["Evaluated entity"] = entity
            row["Reference profile"] = bs_valid[entity]["ref_profile"]
            row["Is valid"] = bs_valid[entity]["conforms"]
            row["Nb errors"] = len(bs_valid[entity]["errors"])
            row["Nb warnings"] = len(bs_valid[entity]["warnings"])
            row["Is the latest profile"] = bs_valid[entity]["latest_profile"]
            row["Is deprecated profile"] = bs_valid[entity]["deprecated"]
            rows.append(row)
            break
    except Exception as e: 
        print(f"Error while validating {ld}")
        print(e)
df2 = pd.DataFrame.from_records(rows)
#df2.to_csv("bioschemas_validation.csv")
df2

  0%|          | 0/136 [00:00<?, ?it/s]

Error while validating https://workflowhub.eu/workflows/18
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while validating https://bio.tools/blast
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error while validating https://zbmed-semtec.github.io/BioMedSem-IR-KD/docs/Lesson1.html
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error while validating https://pathvisio.org/
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while validating http://gigadb.org/site/index
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while val

Error while validating http://phenpath.biocomp.unibo.it/phenpath/
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while validating https://www.guidetopharmacology.org/coronavirus.jsp
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while validating http://159.149.160.88/cscan
[Errno Expecting value] <html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
: 0
Error while validating https://string-db.org/network/9606.ENSP00000379258
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error while validating https://www.chd7.org/
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response

Error while validating https://synbiohub.org/public/igem/BBa_K165005/1
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fsynbiohub.org%2Fpublic%2Figem%2FBBa_K165005%2F1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16060d850>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error while validating https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:37133
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fwww.genenames.org%2Fdata%2Fgene-symbol-report%2F%23%21%2Fhgnc_id%2FHGNC%3A37133 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16060d580>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error while validating https://proteinensemble.org/
HTTPConnectionPool(host='localhost', p

Error while validating http://
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=http%3A%2F%2F (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x112819550>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error while validating https://synbiohub.org/
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fsynbiohub.org%2F (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16060d040>: Failed to establish a new connection: [Errno 61] Connection refused'))
Error while validating https://pippa.psb.ugent.be/pippa_experiments/consult_experiment_basic_info/55
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fpippa.psb.ugent.be%2Fpippa_experiments%2Fc

Error while validating https://workflowhub.eu/
HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fworkflowhub.eu%2F (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1606045b0>: Failed to establish a new connection: [Errno 61] Connection refused'))


""


In [5]:
from IPython.display import display, Markdown, Latex
my_md = '<i class="fa fa-camera"></i>'
display(Markdown(my_md))

<i class="fa fa-camera"></i>

In [6]:
md1 = """
{"@id": "_:N0b2cabb67c8a461a8c1c84367ef89fb1", "http://www.w3.org/1999/xhtml/vocab#role": [{"@id": "http://www.w3.org/1999/xhtml/vocab#img"}]}
"""

md2 = """
{"type": "http://schema.org/Organization", "properties": {"logo": "http://bioschemas.org/assets/img/square_logo2.png", "url": "http://bioschemas.org", "email": "enquiries@bioschemas.org", "name": "Bioschemas Community"}}
"""

md3 = """

"""

In [7]:
#live_deploys_remote_file = "https://raw.githubusercontent.com/BioSchemas/bioschemas.github.io/master/_data/live_deployments.json"
#res = requests.get(live_deploys_remote_file)
#live_deploys = res.json()
#print(json.dumps(live_deploys, indent=True))

In [8]:
mobidb_valid = check_bioschemas("https://mobidb.org")
print(json.dumps(mobidb_valid, indent=True))

ConnectionError: HTTPSConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/inspect/bioschemas_validation_by_conformsto?url=https%3A%2F%2Fmobidb.org (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10b0bee20>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
g = ConjunctiveGraph()
g.parse("http://bioschemas.org/", format="n3")
#print(g.serialize(format="turtle"))

BadSyntax: at line 4 of <>:
Bad syntax (expected '.' or '}' or ']' at end of statement) at ^ in:
"b'<!DOCTYPE html>\n<html lang="en">\n<html>\n  '^b'<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Com'..."